In [1]:
import pandas as pd
df1 = pd.read_csv(u'feature.csv')
df1.head()

,movie_id,movie_name,genres,cast,director,feature
0,3,天堂孤影,"['劇情', '喜劇']",['Matti Pellonpää'],Aki Kaurismäki,天堂孤影 劇情 喜劇 卡德 羅娜 環衛 經理 服裝店 同事 Matti Pellonpää ...
1,11,星際大戰四部曲：曙光乍現,"['冒險', '動作', '科幻']",['Mark Hamill'],George Lucas,星際大戰四部曲曙光乍現 冒險 動作 科幻 銀河 盧克 帝國 機器人 奧爾德 絕地 Mark ...
2,12,海底總動員,"['動畫', '家庭']",['Albert Brooks'],Andrew Stanton,海底總動員 動畫 家庭 馬林 尼莫 卵子 攻擊 潛水員 珊瑚 Albert Brooks A...
3,13,阿甘正傳,"['喜劇', '劇情', '愛情']",['Tom Hanks'],Robert Zemeckis,阿甘正傳 喜劇 劇情 愛情 阿甘 美國 一個 小鎮 愛護 尼克森 Tom Hanks Rob...
4,14,美國心玫瑰情,['劇情'],['Kevin Spacey'],Sam Mendes,美國心玫瑰情 劇情 小珍 卡洛琳 於是 越來越 人生 罕住 Kevin Spacey Sam...


In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# 將文件中的詞語轉換為詞頻矩陣
cv = CountVectorizer()
# 計算個詞語出現的次數
cv_mx = cv.fit_transform(df1['feature'])

# create cosine similarity matrix
cosine_sim = cosine_similarity(cv_mx, cv_mx)
print(cosine_sim)

[[1.         0.         0.         ... 0.08006408 0.07692308 0.        ]
 [0.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.08006408 0.         0.         ... 1.         0.08006408 0.        ]
 [0.07692308 0.         0.         ... 0.08006408 1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [3]:
# create list of indices for later matching
indices = pd.Series(df1.index, index = df1['movie_name'])

In [ ]:
user = pd.read_csv(u'userlist.csv')

df1.drop(['feature'],axis=1,inplace=True)
user = pd.merge(user, df1, on='movie_id')
new = user.movie_name.to_list()
user.head()

for i in new:
    print(i)

In [5]:
def recommend_movie(titles, n = 10, cosine_sim = cosine_sim):
    movies = []
    #for titles in userList
    for title in titles:
        # 檢索匹配的 movie_name index
        if title not in indices.index:
            print("Movie not in database.")
            return
        else:
            idx = indices[title]
        
        # 電影的餘弦相似度分數降序排列
        scores = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
            
        # 前 n 個最相似的 movies indexes
        # 使用 1:n 因為 0 是輸入的同一部電影
        top_n_idx = list(scores.iloc[1:n].index)
                
        #return result
        #print(df1['movie_name'].iloc[top_n_idx])
        ans = df1.iloc[top_n_idx]
        movies.append(ans)
    df = pd.concat(movies)    
    return df

In [6]:
ans = recommend_movie(new,7)
#ans
ans = ans.drop_duplicates(keep='first', inplace=False,subset=['movie_name'])

In [7]:
#清洗數據
from ast import literal_eval
ans['genres'] = ans['genres'].apply(literal_eval)
ans['cast'] = ans['cast'].apply(literal_eval)
ans['cast1'] = ''
ans['genres1'] = ''

user['genres'] = user['genres'].apply(literal_eval)
user['cast'] = user['cast'].apply(literal_eval)
user['cast1'] = ''
user['genres1'] = ''

def genres_words(x):
    return (' ' + ' '.join(x['genres']))
ans['genres1'] = ans.apply(genres_words, axis = 1)
user['genres1'] = user.apply(genres_words, axis = 1)
def cast_words(x):
    return (' ' + ' '.join(x['cast']))
ans['cast1'] = ans.apply(cast_words, axis = 1)
user['genres1'] = user.apply(genres_words, axis = 1)

ans.drop(['genres','cast'],axis=1,inplace=True)
user.drop(['genres','cast'],axis=1,inplace=True)


In [8]:
#genres
featureG = ans.genres1.str.split(' ', expand=True).stack().str.get_dummies().sum(level=0)


/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [9]:
featureGU = user.genres1.str.split(' ', expand=True).stack().str.get_dummies().sum(level=0)


/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


In [10]:
#director
featureDU = user.director.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)


/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [11]:
#director
featureD = ans.director.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)


/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [12]:
#cast
featureC = ans.cast1.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)


/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [13]:
#director
featureCU = user.director.str.split(',', expand=True).stack().str.get_dummies().sum(level=0)


/Users/zhangxinyu/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [14]:
movie_vec = featureG.merge(featureD, how='inner', left_index=True, right_index=True)
movie_vec = movie_vec.merge(featureC, how='inner', left_index=True, right_index=True)
movie_vec = pd.concat([ans,movie_vec], axis=1)
movie_vec.set_index('movie_id',inplace=True)
movie_vec.drop(['genres1','cast1','director','movie_name'],axis=1,inplace=True)
movie_vec
#final = ans.drop_duplicates(keep='first', inplace=False,subset=['movie_name'])

,冒險,劇情,動作,動畫,喜劇,奇幻,家庭,恐怖,愛情,懸疑,...,Elijah Wood,Jeff Bridges,Johnny Depp,Meryl Streep,Michael J. Fox,Mitsuo Iwata,Rumi Hiiragi,Sumi Shimamoto,Tom Hanks,William Shatner
movie_id,,,,,,,,,,,,,,,,,,,,,
12,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
350,0,1,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
129,0,0,0,1,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
105,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
165,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
118,1,0,0,0,1,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
35,0,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
97,1,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
user_vec = pd.DataFrame()
user_vec = user_vec.reindex(columns =movie_vec.columns)
user_vec = user_vec.merge(featureGU, how='outer')
user_vec = user_vec.merge(featureDU, how='outer')
user_vec = user_vec.merge(featureCU, how='outer')
#user_vec = user_vec.merge(featureDU, how='inner', left_index=True, right_index=True)
#user_vec = user_vec.merge(featureCU, how='inner', left_index=True, right_index=True)
user_vec = pd.concat([user, user_vec], axis=1)
user_vec.drop(['genres1','cast1','director','movie_id','movie_name'],axis=1,inplace=True)
user_vec = user_vec.fillna(0.0)
user_vec

,userId,劇情,動作,恐怖,愛情,懸疑,犯罪,驚悚,Andrew Stanton,Chris Kentis,...,Rumi Hiiragi,Sumi Shimamoto,Tom Hanks,William Shatner,冒險,動畫,喜劇,奇幻,家庭,科幻
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
user_vec = user_vec.groupby('userId').mean()
user_vec = user_vec[movie_vec.columns]
user_vec

,冒險,劇情,動作,動畫,喜劇,奇幻,家庭,恐怖,愛情,懸疑,...,Elijah Wood,Jeff Bridges,Johnny Depp,Meryl Streep,Michael J. Fox,Mitsuo Iwata,Rumi Hiiragi,Sumi Shimamoto,Tom Hanks,William Shatner
userId,,,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1.0,0.0,0.0,0.0,1.0,0.5,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,1.0,0.0,0.0,0.5,0.5,0.5,0.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#user movie similar matrix
user_movie_matrix = cosine_similarity(user_vec.values,movie_vec.values)
user_movie_matrix = 1- user_movie_matrix 
user_movie_matrix = pd.DataFrame(user_movie_matrix, index=user_vec.index,columns=movie_vec.index)

In [18]:
import numpy as np
def get_the_most_similar_movies(user_id, user_movie_matrix,num):
 user_vec = user_movie_matrix.loc[user_id].values 
 sorted_index = np.argsort(user_vec)[:num]
 return list(user_movie_matrix.columns[sorted_index])

In [19]:
get_the_most_similar_movies(1, user_movie_matrix,4)

[35, 12, 129, 105]

In [747]:
user_movie_matrix

movie_id,12,350,129,105,165,118,35,149,97,83,257,81,22,121,58,196,13,152,106
userId,,,,,,,,,,,,,,,,,,,
0.0,0.750000,1.000000,0.776393,0.795876,0.795876,1.000000,0.776393,1.000000,1.000000,1.0,1.000000,0.776393,1.000000,1.000000,1.000000,0.776393,0.776393,1.000000,1.000000
1.0,0.333333,0.850929,0.403715,0.591752,0.591752,0.591752,0.254644,0.701858,1.000000,1.0,0.701858,0.701858,1.000000,1.000000,1.000000,0.850929,0.850929,1.000000,1.000000
2.0,0.666667,0.850929,0.552786,0.319586,0.319586,0.319586,0.552786,0.701858,0.552786,1.0,0.850929,0.403715,0.552786,0.552786,0.552786,0.403715,0.850929,0.552786,0.591752
